# Daily Digest: Top Trending Topics for Crude Oil


## Why It Matters
In commodity markets like crude oil, timely awareness of trending topics can directly influence trading strategies, hedging decisions, and market forecasts. As news cycles accelerate, being able to identify not just what is being talked about but also its potential market impact is essential for staying ahead of price-moving developments.

## What It Does
This workflow identifies, verifies, clusters, and summarizes the most relevant and impactful news trends in the crude oil market using the Bigdata API for retrieval and large language models for topic analysis. It produces both daily market reports and structured datasets that can be used for monitoring or backtesting.

## How It Works
The notebook implements a four-step agentic workflow built on Bigdata API:

- **Lexicon Generation** of industry-specific jargon to maximize recall in news retrieval.

- **Content Retrieval** via the Bigdata API, splitting searches into daily windows and parallelizing keyword lookups for speed.

- **Topic Clustering & Selection** to verify, group, and summarize news into ranked trending topics, scoring each for trendiness, novelty, impact, and magnitude.

- **Custom Report Generation** in the form of a daily digest with a configurable ranking system, supported by granular news sources for verification.

## A Real-World Use Case 
This cookbook illustrates the full workflow through a practical example: tracking top trending topics in the crude oil market during the Israel-Iran tensions of June 2025. You'll learn how to transform unstructured news into structured, ranked insights on market-moving narratives.


## Setup and Imports

## Async Compatibility Setup

**Run this cell first** - Required for Google Colab, Jupyter Notebooks, and VS Code with Jupyter extension:

### Why is this needed?

Interactive environments (Colab, Jupyter) already have an asyncio event loop running. When bigdata-research-tools makes async API calls (like to OpenAI), you'll get this error without nest_asyncio:

```
RuntimeError: asyncio.run() cannot be called from a running event loop
```

The `nest_asyncio.apply()` command patches this to allow nested event loops.

💡 **Tip**: If you're unsure which environment you're in, just run the cell below - it won't hurt in any environment!

In [2]:
import datetime
start = datetime.datetime.now()

try:
    import asyncio
    asyncio.get_running_loop()
    import nest_asyncio; nest_asyncio.apply()
    print("✅ nest_asyncio applied")
except (RuntimeError, ImportError):
    print("✅ nest_asyncio not needed or not available")

✅ nest_asyncio applied


## Environment Setup

The following cell configures the necessary path for the analysis

In [3]:
import os
import sys


current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.append(current_dir)
print(f"✅ Local environment setup complete")

✅ Local environment setup complete


## Import Required Libraries

Import the core libraries needed for trending topics analysis, including the lexicon generation classes and data processing tools.

In [4]:
from bigdata_client import Bigdata
from bigdata_client.models.search import DocumentType
from src.lexicon_generator import LexiconGenerator
from src.search_topics import search_by_keywords
from src.topics_extractor import (process_all_reports,
                                run_process_all_trending_topics,
                                run_add_advanced_novelty_scores,
                                add_market_impact_to_df,
                                prepare_data_for_report,
                                generate_html_report)
from IPython.display import display
from IPython.core.display import HTML

## Define Output Paths

We define the output paths for our thematic screening results.

In [5]:
# Define output file paths for our report
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

export_path = f"{output_dir}/trending_topics_crude_oil_results.xlsx"

## Load Credentials

In [6]:

from dotenv import load_dotenv
from pathlib import Path

script_dir = Path(__file__).parent if '__file__' in globals() else Path.cwd()
#load_dotenv(script_dir / '.env')
load_dotenv(os.path.abspath("/home/abouchs/.python_env_var/.env"))

BIGDATA_USERNAME = os.getenv('BIGDATA_USERNAME')
BIGDATA_PASSWORD = os.getenv('BIGDATA_PASSWORD')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

if not all([BIGDATA_USERNAME, BIGDATA_PASSWORD, OPENAI_API_KEY]):
    print("❌ Missing required environment variables")
    raise ValueError("Missing required environment variables. Check your .env file.")
else:
    print("✅ Credentials loaded from .env file")

✅ Credentials loaded from .env file


## Connecting to Bigdata

Create a Bigdata object with your credentials.

In [7]:
bigdata = Bigdata(BIGDATA_USERNAME, BIGDATA_PASSWORD)

## Defining Your Trending Topics Context and Parameters

To perform a trending topics analysis, we need to define a few key parameters:

- **Main Theme** (`main_theme`): The main topic, asset class, or context to analyze (e.g. Crude Oil)

- **Time Period** (`start_date` and `end_date`): The date range over which to run the search

- **Frequency** (`freq`): The frequency at which you want to produce a daily digest

- **Document Type** (`document_type`): Specify which documents to search over (transcripts, filings, news)

- **Model Selection** (`llm_model`): The AI model used for semantic analysis and topic classification

In [18]:
# ===== Context Definition =====
main_theme = 'Crude Oil'

# ===== Specify Time Range =====
start_query = '2025-06-21'
end_query = '2025-06-28'

# ===== Query Configuration =====
document_type = DocumentType.NEWS

# ===== LLM Specification =====
llm_model = "gpt-4o-mini"

## Instantiating the Lexicon Generator

In this step, we identify the specialized industry-specific jargon relevant to the crude oil market to ensure a high recall in the content retrieval.

In [9]:
LexiconGenerator = LexiconGenerator(openai_key=OPENAI_API_KEY, model="gpt-4o", seeds=[123, 123456, 123456789, 456789, 789])

In [10]:
keywords = LexiconGenerator.generate(theme=main_theme)

## Content Retrieval from Bigdata Search API

In this section, we perform a keyword search on the news content with the Bigdata API to retrieve documents, splitting the search over daily timeframes and multi-threading the content search on the individual keywords for speed purpose. The user can define the time range below to generate daily reports between the start and end dates.

With the list of market-specific keywords parameters, you can leverage the Search functionalities in bigdata-research-tools, built with Bigdata API, to run search at scale against news documents. We need to define 2 more parameters for search:

- **Frequency** (`freq`): The frequency of the date ranges to search over. It should at least match the frequency of your daily digest. Supported values:
    - `Y`: Yearly intervals.
    - `M`: Monthly intervals.
    - `W`: Weekly intervals.
    - `D`: Daily intervals. Defaults to `D`.
- **Document Limit** (`document_limit`): The maximum number of documents to return per query to Bigdata API.

In [13]:
# ===== Query Configuration =====
document_limit = 10 # Maximum number of documents to retrieve per query
frequency = 'D'  # Query frequency

In [14]:
results, daily_keyword_count = search_by_keywords(
    keywords=keywords,
    start_date=start_query,
    end_date=end_query,
    scope = document_type,
    freq=frequency,
    document_limit=document_limit)

About to run 856 queries
Example Query: Keyword('Crude Oil') over date range: AbsoluteDateRange('2025-06-21T00:00:00', '2025-06-21T23:59:59')


Querying Bigdata...: 100%|██████████| 856/856 [02:58<00:00,  4.79it/s]


## Topic Clustering and Summarization

In this step, we perform topic modelling using a large language model to verify and cluster the news. Then, the summarization ensures topic selection identifying the top trending news for crude oil, while deriving advanced analytics to quantify the trendiness (based on news volume), novelty (based on daily changes in summaries), impact and magnitude (based on the financial materiality on crude oil prices) of the trending topics.

Before performing the topic clustering, we apply a verification layer to remove the news that are not relative to the oil market

In [19]:
semaphore_size = 1000 # Maximum number of concurrent requests to Openai API

# Assuming unique_reports is your DataFrame
filtered_reports = process_all_reports(results, llm_model, OPENAI_API_KEY, main_theme, semaphore_size)

Filtering News:   0%|          | 0/6961 [00:00<?, ?it/s]

In [20]:
filtered_reports

,timestamp,rp_document_id,headline,chunk_number,sentence_id,source_id,source_name,text,keyword,date
0,2025-06-21 00:00:00+00:00,0B4844B85EA092ED4AB50AB3534AB360,BW Energy: Update on Fixed Income Investor Mee...,3.0,0B4844B85EA092ED4AB50AB3534AB360-3,923B93,Financial Services Monitor Worldwide,BW Energy is a growth E&P company with a diffe...,Barrel,2025-06-21
1,2025-06-21 00:00:00+00:00,958293ED6CDCFE1D42FE513A05B3A7DA,"Geothermal Drill Bits Market Trends, Opportuni...",7.0,958293ED6CDCFE1D42FE513A05B3A7DA-7,923B93,Financial Services Monitor Worldwide,Key Market Trends: Adoption of Advanced Materi...,Drilling,2025-06-21
2,2025-06-21 00:00:00+00:00,EE3309C79EA552874F436912D2F6A67A,US Supreme Court sets rules for venue selectio...,4.0,EE3309C79EA552874F436912D2F6A67A-4,BC923D,Legal Monitor Worldwide,"Here, the Supreme Court held that the nationwi...",Refinery,2025-06-21
3,2025-06-21 00:00:00+00:00,958293ED6CDCFE1D42FE513A05B3A7DA,"Geothermal Drill Bits Market Trends, Opportuni...",5.0,958293ED6CDCFE1D42FE513A05B3A7DA-5,923B93,Financial Services Monitor Worldwide,Key Market Challenges: Geological and Environm...,Drilling,2025-06-21
4,2025-06-21 00:00:00+00:00,958293ED6CDCFE1D42FE513A05B3A7DA,"Geothermal Drill Bits Market Trends, Opportuni...",2.0,958293ED6CDCFE1D42FE513A05B3A7DA-2,923B93,Financial Services Monitor Worldwide,"Moreover, government incentives, along with ex...",Drilling,2025-06-21
...,...,...,...,...,...,...,...,...,...,...
3671,2025-06-28 23:40:24+00:00,5DF2066D742E8ADBE14FFB5341CECFEE,Chennai airport conducts full scale emergency ...,1.0,5DF2066D742E8ADBE14FFB5341CECFEE-1,80FC03,The Times Of India,"Chennai: An aircraft engulfed in flames, fire ...",Drilling,2025-06-28
3672,2025-06-28 23:40:24+00:00,5DF2066D742E8ADBE14FFB5341CECFEE,Chennai airport conducts full scale emergency ...,2.0,5DF2066D742E8ADBE14FFB5341CECFEE-2,80FC03,The Times Of India,The drill caused a traffic snarl on the GST ro...,Drilling,2025-06-28
3673,2025-06-28 23:40:24+00:00,5DF2066D742E8ADBE14FFB5341CECFEE,Chennai airport conducts full scale emergency ...,3.0,5DF2066D742E8ADBE14FFB5341CECFEE-3,80FC03,The Times Of India,The entire exercise was monitored and evaluate...,Drilling,2025-06-28
3674,2025-06-28 23:48:49+00:00,456DB9D74C8F0B01626EE845F8FF4CA6,Caught on camera: Car literally drives through...,2.0,456DB9D74C8F0B01626EE845F8FF4CA6-2,E54C73,ABC News,"""This is like a movie or something,"" Patel sai...",Barrel,2025-06-28


In this cell, we leverage a LLM to perform topic modeling, identifying and clustering the key topics from the news reports.

In [21]:
flattened_trending_topics_df = run_process_all_trending_topics(
    unique_reports=filtered_reports,
    model=llm_model,
    start_query=start_query,
    end_query=end_query,
    api_key=OPENAI_API_KEY,
    main_theme=main_theme,
    batches=20 #number of batches to process the reports in parallel (adjust based on your system's capabilities)
)


Extracting Topics for 2025-06-21:   0%|          | 0/20 [00:00<?, ?it/s]

Extracting Topics for 2025-06-22:   0%|          | 0/20 [00:00<?, ?it/s]

Extracting Topics for 2025-06-23:   0%|          | 0/20 [00:00<?, ?it/s]

Extracting Topics for 2025-06-24:   0%|          | 0/20 [00:00<?, ?it/s]

Extracting Topics for 2025-06-25:   0%|          | 0/20 [00:00<?, ?it/s]

...

Consolidating topic batches: 100%|██████████| 10/10 [00:23<00:00,  2.36s/it]


Summarizing text for each topic...


Generating titles: 100%|██████████| 24/24 [00:07<00:00,  3.41it/s]


Generating Day in Review summaries...
Adding one-line summaries to DataFrame...


Generating text summaries: 100%|██████████| 1623/1623 [01:05<00:00, 24.95it/s] 


## Topic Scoring

**Trendiness and Novelty Scores**: We derive analytics related to the trendiness of the topic based on the news volume, and the novelty of the topic based on the changes in daily summaries, evaluating the uniqueness and freshness of each topic.

In [22]:
# Calculate trendiness and novelty scores, assessing the uniqueness and freshness of each topic
flattened_trending_topics_df = run_add_advanced_novelty_scores(flattened_trending_topics_df, api_key = os.environ['OPENAI_API_KEY'], main_theme = main_theme)

Calculating Novelty Scores:   0%|          | 0/54 [00:00<?, ?it/s]

**Financial Materiality**: We derive analytics related to the impact (Positive, Negative) and magnitude (High, Medium, Low) of the topics, inferring their  market impact on crude oil prices. The inference is based on the price mechanisms involving supply and demand dynamics, geopolitical factors among others.

In [23]:
point_of_view = "a crude oil trader, where price is influenced by supply-demand dynamics, geopolitical events, and market sentiment. \
As a trader, you focus on changes in production, inventories, and economic indicators from key markets."

flattened_trending_topics_df = add_market_impact_to_df(flattened_trending_topics_df, api_key = os.environ['OPENAI_API_KEY'], main_theme = main_theme, point_of_view = point_of_view)

We display the results of topic modeling and summarization. The **Topic** column represents the themes inferred through topic clustering using a LLM, which groups the news articles based on their content and underlying themes. The **Summary** provides a synthesized overview of all news articles within the same topic, offering a high-level view of the key messages for each cluster. The **Topic** is then rephrased into a concise form based on the summary. The **Text_Summary** provides a detailed summary of each individualchunk, capturing its core message.

In [24]:
flattened_trending_topics_df.head(5)

,Date,Day_in_Review,Topic,Summary,Source,Headline,Text,Volume_Score,Text_Summary,Volume_Score,Novelty_Score,Impact_Score,Magnitude_Score
0,2025-06-21,- **US-Iran Conflict Escalation**: Heightened ...,US-Iran Conflict Escalation Drives Volatility ...,"The US-Iran conflict, particularly in the cont...",The Economic Times,Oil prices fall as US delays decision on direc...,Oil prices fell on Friday after the White Hous...,4,Oil prices declined due to delayed U.S. involv...,4,New,Positive,High
3,2025-06-21,- **US-Iran Conflict Escalation**: Heightened ...,US-Iran Conflict Escalation Drives Volatility ...,"The US-Iran conflict, particularly in the cont...",Asharq Al-Awsat,Water Levels Plummet at Drought-Hit Iraqi Rese...,'Harvest failed' -\nUpstream damming of the Li...,4,Iran's upstream damming of the Little Zab Rive...,4,New,Positive,High
4,2025-06-21,- **US-Iran Conflict Escalation**: Heightened ...,Iran-Israel Conflict Fuels Oil Price Surge Ami...,"Geopolitical tensions in the Middle East, part...",Yahoo! Finance,How a contrarian trade can help you as an inve...,As President Donald Trump decides whether or n...,5,The potential U.S. involvement in Israel's mil...,5,Old,Positive,High
5,2025-06-21,- **US-Iran Conflict Escalation**: Heightened ...,Iran-Israel Conflict Fuels Oil Price Surge Ami...,"Geopolitical tensions in the Middle East, part...",EnergyNow,Oil Prices Settle Lower as US Sanctions Ease F...,Oil prices jumped almost 3% on Thursday after ...,5,Escalating military actions between Israel and...,5,Old,Positive,High
6,2025-06-21,- **US-Iran Conflict Escalation**: Heightened ...,Iran-Israel Conflict Fuels Oil Price Surge Ami...,"Geopolitical tensions in the Middle East, part...",Time Magazine,What Conflict in the Middle East Means for Cli...,Central to that picture are oil prices. In the...,5,Rising tensions in the Iran-Israel conflict ha...,5,Old,Positive,High


For verification purpose, this actionable timestamped dataframe contains the granular news clustered into relevant topics, and also the advanced analytics of trendiness, novelty, impact, and magnitude scores to be potentially used for backtesting.

## Generate a Custom Daily Digest

In this step, we rank the topics, allowing the user to customize the ranking system to reindex the news, based on their trendiness, novelty, and financial materiality on crude oil prices. We finally display a daily market update, supported by the corresponding granular news and sources for verification purpose.

The user selects the date for the report summarizing the top trending topics, and customizes the ranking system to prioritize the topics based on volume (trendiness and media attention), novelty (based on the emergence of new daily news), impact direction (positive or negative), and magnitude (financial materiality). The ranking system prioritizes the criteria in the order specified by the user, allowing for a tailored focus on the most relevant aspects of the data.

The order in which the criteria are listed in user_selected_ranking determines their priority for ranking the topics within the report. The first criterion in the list has the highest priority, followed by the second, and then the third. The user can customize the ranking by choosing to prioritize impact direction (positive or negative), novelty, magnitude, or volume, and has the flexibility to select 1, 2, or all 3 criteria based on their specific needs.

In [25]:
specific_date = '2025-06-23'  # Example date, can be modified as needed
user_selected_ranking = ['novelty', 'volume', 'magnitude']  # User can modify this list to change the ranking order
#impact_filter = 'positive_impact' #User can use the impact_filter to filter out the report

In [26]:
prepared_reports = prepare_data_for_report(flattened_trending_topics_df, user_selected_ranking, impact_filter = None, report_date = specific_date)

In [27]:
# Generate and display the HTML report for each date
for report in prepared_reports:
    html_content = generate_html_report(
        report['date'],
        report['day_in_review'],
        report['topics'],
        'Daily crude oil market update'  # Pass the main theme to dynamically generate the title
    )
    display(HTML(html_content))
    print("")
    print("")
    print("")